In [1]:
import numpy as np

In [2]:
def write_header_lines(file, layer_height, line_width, layer_count, mesh, start_x, start_y, start_z=0.3):
    with open(file, 'w') as f:
        header_lines = [
            f";layer_height = {layer_height}",
            f"\n;line_width = {line_width}",
            f"\n;layer_count = {layer_count}",
            f"\n;mesh = {mesh}"
        ]
        
        initialize_lines = [
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n",
            f"\nG0 F3600 X{start_x:.3f} Y{start_y:.3f} Z{start_z:.3f} ;Go to start position",
            "\nM7",
            "\n\n"
        ]
        
        f.writelines(header_lines)
        f.writelines(initialize_lines)
        
        
def writ_finish_lines(file):
    with open(file, 'a') as f:
        finish_lines = [
            "\n\n;Finish",
            "\nM9",
            "\nG28 X-100 Y-100;Home",
        ]
        f.writelines(finish_lines)

In [3]:
def write_G1_line(delta_x, delta_y, xy, lines, pause=True):
    xy += [delta_x, delta_y]
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")
    if pause:
        lines.append(f"\nG4 P60")
    # print(xy)
    return xy, lines

def write_init_layer(xy, z, lines):
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f} Z{z:.3f}")
    return lines    

In [13]:
def draw_layer_out_in(file, xy1, xy3, z, w, n):
    """
    Args:
        n (int): wall count
    """
    
    xy = np.array([xy1[0] + w/2, xy1[1] + w/2])
    a = xy3[0] - xy1[0] - w
    b = xy3[1] - xy1[1] - w
    
    lines = []
    lines = write_init_layer(xy, z, lines)
    for i in range (n):
        xy, lines = write_G1_line(a-w*i*2, 0, xy, lines)
        xy, lines = write_G1_line(0, b-w*i*2, xy, lines)
        xy, lines = write_G1_line(-a+w*i*2, 0, xy, lines)
        xy, lines = write_G1_line(0, -b+w*i*2, xy, lines)
        if i != n-1:
            xy, lines = write_G1_line(w, w, xy, lines, pause=False)
        
    with open(file, 'a') as f:
        f.writelines(lines)
    
        
def draw_layer_in_out(file, xy1, xy3, z, w, n):
    """
    Args:
        n (int): wall count
    """
    
    xy = np.array([xy1[0]+w/2+w*(n-1), xy1[1]+w/2+w*(n-1)])
    a = xy3[0] - xy1[0] - w
    b = xy3[1] - xy1[1] - w
    
    lines = []
    lines = write_init_layer(xy, z, lines)
    for i in range (n):
        xy, lines = write_G1_line(a-w*(n-i-1)*2, 0, xy, lines)
        xy, lines = write_G1_line(0, b-w*(n-i-1)*2, xy, lines)
        xy, lines = write_G1_line(-a+w*(n-i-1)*2, 0, xy, lines)
        xy, lines = write_G1_line(0, -b+w*(n-i-1)*2, xy, lines)
        if i != n-1:
            xy, lines = write_G1_line(-w, -w, xy, lines, pause=False)
        
    with open(file, 'a') as f:
        f.writelines(lines)

In [17]:
layer_height = 0.3
line_width = 0.4
mesh = 'side'
length_a = 21.6
length_b = 5.6
wall_layer_count = 2
layer_count = int(12/layer_height)
file = '7.23/walls4_'+str(wall_layer_count)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -length_a/2, -length_b/2)

for i in range (layer_count):
    xy1 = np.array([-length_a/2, -length_b/2])
    xy3 = np.array([length_a/2, length_b/2])
    z = i * layer_height + 0.3
    
    if i % 2 == 0:
        draw_layer_out_in(file, xy1, xy3, z, line_width, wall_layer_count)
    else:
        draw_layer_in_out(file, xy1, xy3, z, line_width, wall_layer_count)
        
writ_finish_lines(file)